In [1]:
import os
import bisect
import itertools
import math
import numpy as np
import pandas as pd
import requests
import pickle
from datetime import datetime as dt
from datetime import timedelta
import pytz
from itertools import zip_longest
from pytz import timezone
from pprint import pprint
import pandas_datareader.data as web
from collections import namedtuple, Counter
from operator import itemgetter

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import fundamentalanalysis as fa
import streamlit as st

import holidays
import pandas_datareader.data as web
import yahoo_fin.stock_info as si
from urllib.request import Request, urlopen
from html_table_parser.parser import HTMLTableParser 
import streamlit as st

from load_data import *
from get_data import get_tickers
from info import TA_PERIODS

pd.options.plotting.backend = "plotly"
# pd.set_option('display.max_rows', None)

In [2]:
rf_rates = get_rf_data()
SPY_df = get_SPY_data()
SPY_info_df = get_SPY_info()
ticker_info = get_ticker_info()
ticker_list = SPY_info_df.index.to_list()
sector_list = SPY_info_df['Sector'].unique()
first_date = SPY_df.iloc[0].name
last_date = SPY_df.iloc[-1].name
yr_ago = last_date - timedelta(days=365)
# TTM_ratios, ratios_data_report = load_TTM_ratios()

# Technical Analysis

#### To Do:

- Trend Analysis
- MAs
- Volume Analysis

## Trend Analysis

In [3]:
from scipy.signal import find_peaks, peak_prominences, argrelextrema, argrelmax, argrelmin
from sklearn import preprocessing

pd.options.display.float_format = "{:,.2f}".format

In [4]:
# end = last_date
# years = 1
# days = 180
# start = end - timedelta(days * years)
# ticker = 'a'
# df = get_ticker_data(ticker)[start:end]
# df_5m = resample_data(ticker, '5m')
# # start = end - timedelta(20)
# # print(divmod((end - start).days, 365))
# print(f'start: {start.date()}, end: {end.date()}')

### The Definition & Interaction of Trends

A trend is a period in which a price moves in an irregular but persistent direction.
The three most widely followed
trends are primary, intermediate, and short-term.

__Primary__

The primary trend generally lasts between 9 months and 2 years, and is
a reflection of investors’ attitudes toward unfolding fundamentals in the
business cycle. The business cycle extends statistically from trough to
trough for approximately 3.6 years, so it follows that rising and falling
primary trends (bull and bear markets) last for 1 to 2 years. Since building 
up takes longer than tearing down, bull markets generally last longer
than bear markets. The direction of the secular or very long-term trend
will also affect the magnitude and duration of a primary trend. Those that
move in the direction of the secular trend will generally experience greater
magnitude and duration than those that move in the opposite direction.

__Intermediate__

Anyone who has looked at prices on a chart will notice that they do not move
in a straight line. A primary upswing is interrupted by several re actions
along the way. These countercyclical trends within the confines of a primary
bull market are known as intermediate price movements. They last anywhere
from 6 weeks to as long as 9 months, sometimes even longer, but rarely
shorter. Countercyclical intermediate trends are typically very deceptive,
often being founded on very believable but false assumptions. For example, 
an intermediate rally during a bear market in equities may very well be
founded on a couple of unexpectedly positive economic numbers, which
make it appear that the economy will avoid that much-feared recession.
When subsequent numbers are reported and found to be wanting, the bear
market resumes.
It is important to have an idea of the direction and maturity of the
primary trend, but an analysis of intermediate trends is also helpful for
improving success rates in trading, as well as for determining when the
primary movement may have run its course.

__Short-Term Trends__

Short-term trends typically last 3 to 6 weeks, sometimes shorter and sometimes 
longer. They interrupt the course of the intermediate cycle, just as
the intermediate-term trend in terrupts primary price movements. Short-
term trends are shown in the market cycle model (Figure 1.1) as a dashed
line. They are usually influenced by random news events and are far more
difficult to identify than their intermediate or primary counterparts.



__Major Technical Principles__
- The direction of the primary trend will affect the 
character of intermediate and short-term trends.
- Surprises occur in the direction of the main trend, i.e., on the upside in a 
bull market and on the downside in a bear market.
- Never go for perfection; always shoot for consistency.
- The significance of a peak-and-trough
reversal is determined by the du ration and magnitude of the rallies and
reactions in question.


Maybe use linreg gradient to determine primary and secondary trend

In [9]:
# dt.strftime(end, '%d/%m/%y %H:%M')

'17/03/23 00:00'

In [11]:
def peaks_valleys_analysis(df):
    df = df.copy().drop(columns='Adj Close')
    close = df['Close']
    peaks = argrelextrema(close.to_numpy(), np.greater)[0]
    valleys = argrelextrema(close.to_numpy(), np.less)[0]
    df['isPeak'] = 0
    df['isValley'] = 0
    df['PV Trend'] = 0
    PV = sorted(list(peaks) + list(valleys))
    first = 'Peak' if min(peaks[0], valleys[0]) == peaks[0] else 'Valley'
    second = 'Peak' if first == 'Valley' else 'Valley'
    i, j = (0, 1)
    d0 = abs(close[PV[0]] - close[0])
    dist = [d0]   
    valid_PV = [PV[0]]
    print(f'i: {i}, j: {j}, {close[PV[0]]:.2f} - {close[0]:.2f} = {d0:.2f}\n')
    
    while i < len(PV):
        col = first if i % 2 == 0 else second
        ix = df.iloc[PV[i]].name
        # df.loc[ix, f'is{col}'] = 1
        
        if i == 0:
            d1 = abs(close[PV[0]] - close[0])
            dist.append(d1)
            d2 = abs(close[PV[j]] - close[PV[i]])
            print(f'{close[PV[j]]:.2f} - {close[PV[i]]:.2f} = {d2:.2f}',
                f'\n{close[PV[0]]:.2f} - {close[0]:.2f} = {d1:.2f}',
                f'\n{d2:.2f} / {d1:.2f} = {d2 / d1:.2f}')
        else:
            d2 = abs(close[PV[j]] - close[PV[i]])
            print(f'{close[PV[j]]:.2f} - {close[PV[i]]:.2f} = {d2:.2f}',
                #   f'\n{close[PV[j]]:.2f} - {close[valid_PV[-1]]:.2f} = {d1:.2f}',
                f'\n{d2:.2f} / {d1:.2f} = {d2 / d1:.2f}')
            
        
        
        retracement = d2 / d1
        if retracement >= 1/3:
            d1 = abs(close[PV[j]] - close[valid_PV[-1]])
            dist.append(d1)
            if j > 1:
                valid_PV.append(PV[j])
                
            df.loc[ix, f'is{col}'] = 1
            df.loc[ix, col] = close[PV[j]]
            if i < len(PV) and j < len(PV):
                j += 1      
                i = j - 1
            print(f'i: {i}, j: {j}, {col}\n{valid_PV}\n')
        else:
            if i < len(PV) and j < len(PV):
                i = PV.index(valid_PV[-1])
                j += 1
                print(f'i: {i}, j: {j}, {col}\n{valid_PV}\n')
                # if len(valid_PV) > 1:
                    # j = valid_PV[-1]
                valid_PV.append(PV[i])
                invalid = valid_PV.pop()
                
                df.loc[ix, f'is{col}'] = 0
                df.loc[ix, col] = 0
                print(f'{ix.date()}: {col} - {close[invalid]:.2f} removed \n') 
            
        # d1 = abs(close[PV[i]] - close[valid_PV[-1]])
            # a, b = (i, i - 1) if i % 2 != 0 else (i, i)
            
    
            # print(f'{i}: {close[PVs[b][n]]:.2f} - {close[PVs[a][m]]:.2f} = {d2:.2f},\
            #        \nRetraced: {d2:.2f} / ({close[pv]:.2f} - {close[p1]:.2f})  = {d2/d1:.2f}\n')
            
            # m, n = n, m
            # d_ix1 = df.iloc[max(PVs[i - 1])].name
            # d_ix2 = df.iloc[min(PVs[i])].name
            # df.loc[d_ix1, 'PV Dist'] = d1
            # df.loc[d_ix2, 'PV Dist'] = d2
            # df.loc[d_ix2, 'Retraced'] = d2 / d1
            # if close[p] - close[p1] >= 0 and close[v] - close[pv] >= 0:
            #     trend = 1 # uptrend
            # elif close[p] - close[p1] < 0 and close[v] - close[pv] < 0:
            #     trend = -1 # downtrend
            # else:
            #     trend = 0
            # start = df.iloc[min(p1, pv)].name
            # end = df.iloc[max(p, v)].name
            # df.loc[start:end, 'PV Trend'] = trend
            # print(f"Peak:   {dt.strftime(df.iloc[p1].name, '%d-%m-%y')} - {dt.strftime(ix, '%d-%m-%y')} " \
            #   f"|| {close[p1]:,.2f} - {close[p]:,.2f},\n" \
            #   f"Valley: {dt.strftime(df.iloc[pv].name, '%d-%m-%y')} - {dt.strftime(ix, '%d-%m-%y')} " \
            #   f"|| {close[pv]:,.2f} - {close[v]:,.2f},\n" \
            #   f"Trend: {trend}\n")

    # CAN USE CHANGES IN TREND TO IDENTIFY POINTS TO DRAW TRENDLINES BTWN
    # df['PV Changepoint'] = df['PV Trend'].diff()
    # ixs = df[df['PV Changepoint'] != 0].index.to_list()
    trendlines = []
    
    # for i, ix in enumerate(ixs[1:], 1):
    #     start = ixs[i - 1]
    #     end = ix
    #     npeaks = df.loc[start:end, 'isPeak'].sum()
    #     nvalleys = df.loc[start:end, 'isValley'].sum()
    #     indices = []
    #     if df.loc[ix, 'PV Changepoint'] < 0 and npeaks > 2:
    #         indices = df[df['isPeak'] != 0][start:end].index.to_list()
    #     if df.loc[ix, 'PV Changepoint'] > 0 and nvalleys > 2:
    #         indices = df[df['isValley'] != 0][start:end].index.to_list()
    #     if indices:    
    #         x1, x2 = indices[0], indices[-1]
    #         y1, y2 = close[x1], close[x2]
    #         trendlines.append(((x1, x2), (y1, y2)))
    #     # print(f"{i}.".ljust(3), f"{dt.strftime(start, '%d-%m-%y')} - {dt.strftime(end, '%d-%m-%y')}, peaks: {peaks.sum()}, valleys: {valleys.sum()}")
    
    # cols = ['isPeak', 'isValley', 'Peak', 'Valley', 'PV Dist', 'Retraced', 'PV Changepoint']
    # df.drop(columns=cols, inplace=True)
    
    return df, peaks, valleys, trendlines

In [12]:
pv_df, peaks, valleys, trendlines = peaks_valleys_analysis(df)
# print(trendlines)

i: 0, j: 1, 121.61 - 130.72 = 9.11

123.72 - 121.61 = 2.11 
121.61 - 130.72 = 9.11 
2.11 / 9.11 = 0.23
i: 0, j: 2, Valley
[6]

2022-09-27: Valley - 121.61 removed 

121.55 - 121.61 = 0.06 
121.61 - 130.72 = 9.11 
0.06 / 9.11 = 0.01
i: 0, j: 3, Valley
[6]

2022-09-27: Valley - 121.61 removed 

132.64 - 121.61 = 11.03 
121.61 - 130.72 = 9.11 
11.03 / 9.11 = 1.21
i: 3, j: 4, Valley
[6, 12]

125.64 - 132.64 = 7.00 
7.00 / 11.03 = 0.63
i: 4, j: 5, Peak
[6, 12, 16]

127.90 - 125.64 = 2.26 
2.26 / 7.00 = 0.32
i: 4, j: 6, Valley
[6, 12, 16]

2022-10-11: Valley - 125.64 removed 

125.70 - 125.64 = 0.06 
0.06 / 7.00 = 0.01
i: 4, j: 7, Valley
[6, 12, 16]

2022-10-11: Valley - 125.64 removed 

132.30 - 125.64 = 6.66 
6.66 / 7.00 = 0.95
i: 7, j: 8, Valley
[6, 12, 16, 21]

125.94 - 132.30 = 6.36 
6.36 / 6.66 = 0.95
i: 8, j: 9, Peak
[6, 12, 16, 21, 23]

137.69 - 125.94 = 11.75 
11.75 / 6.36 = 1.85
i: 9, j: 10, Valley
[6, 12, 16, 21, 23, 27]

136.49 - 137.69 = 1.20 
1.20 / 11.75 = 0.10
i: 9, j: 11, Pe

IndexError: list index out of range

### Trend Changepoints

In [4]:
def order_peaks_valleys_plot(df, P, V, first):
    '''Remove smallest peak if peaks not followed by valleys & vice versa'''
    
    lst = P if first == 'P' else V
    olst = P if first != 'P' else V
    fn = min if first == 'P' else max
    removed_vals = []

    print("\tS: order_peaks_valleys".upper())
    # print(lst)
    # print(olst)
    
    for i, (p, v) in enumerate(zip(P, V)):
        a = p if first == 'P' else v
        b = v if first == 'P' else p

        if i < len(lst) - 1:
            c = P[i + 1] if first == 'P' else V[i + 1]
            if c < b:
                val = fn((a, df.Close[a]), (c, df.Close[c]), key=itemgetter(1))[0]
                lst.remove(val)
                removed_vals.append(val)
                print(f'\ti: {i}, {df.index[val]} ({val}, {df.Close[val]:.2f})')

    # print(lst)
    # print(olst)
    print(f'\n\tfirst: {first}, peaks: {len(P)}, valleys: {len(V)}')
    print("\tE: order_peaks_valleys".upper())
    
    return removed_vals

In [5]:
def order_peaks_valleys_debug(df, P, V, first):
    '''Remove smallest peak if peaks not followed by valleys & vice versa'''
    
    lst = P if first == 'P' else V
    olst = P if first != 'P' else V
    fn = min if first == 'P' else max

    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}\n')
    print(lst)
    print(olst)
    
    for i, (p, v) in enumerate(zip(P, V)):
        a = p if first == 'P' else v
        b = v if first == 'P' else p

        if i < len(lst) - 1:
            c = P[i + 1] if first == 'P' else V[i + 1]
            if c < b:
                val = fn((a, df.Close[a]), (c, df.Close[c]), key=itemgetter(1))[0]
                lst.remove(val)
                print(f'i: {i}, {df.index[val]} ({val}, {df.Close[val]:.2f})')

    print(f'\nfirst: {first}, peaks: {len(P)}, valleys: {len(V)}')
    print(lst)
    print(olst)
    
    return P, V

In [6]:
def order_peaks_valleys(df, P, V, first):
    '''Remove smallest peak if peaks not followed by valleys & vice versa'''
    
    lst = P if first == 'P' else V
    fn = min if first == 'P' else max
    
    for i, (p, v) in enumerate(zip(P, V)):
        a = p if first == 'P' else v
        b = v if first == 'P' else p

        if i < len(lst) - 1:
            c = P[i + 1] if first == 'P' else V[i + 1]
            if c < b:
                val = fn((a, df.Close[a]), (c, df.Close[c]), key=itemgetter(1))[0]
                lst.remove(val)
    
    return P, V

In [7]:
period = '30m'
days = TA_PERIODS[period]['days']
end = last_date
start = end - timedelta(days)

df = make_dataframe('a', period)[start:end]
close = df.Close
P = argrelextrema(close.to_numpy(), np.greater)[0].tolist() # peaks
V = argrelextrema(close.to_numpy(), np.less)[0].tolist() # valleys
first = 'P' if P[0] < V[0] else 'V'

order_peaks_valleys(df, P, V, first)

([1,
  6,
  8,
  14,
  16,
  21,
  24,
  28,
  31,
  33,
  38,
  42,
  47,
  49,
  52,
  59,
  63,
  65,
  70,
  75,
  77,
  81,
  83,
  89,
  95,
  100,
  104,
  106,
  109,
  116,
  120,
  127,
  130,
  133,
  138,
  140,
  144,
  146,
  152,
  155,
  158,
  163,
  166,
  168,
  172,
  175],
 [3,
  7,
  11,
  15,
  18,
  23,
  26,
  29,
  32,
  37,
  39,
  43,
  48,
  50,
  53,
  60,
  64,
  69,
  74,
  76,
  80,
  82,
  88,
  92,
  99,
  101,
  105,
  108,
  115,
  118,
  125,
  129,
  131,
  136,
  139,
  141,
  145,
  150,
  154,
  157,
  161,
  164,
  167,
  170,
  174,
  176])

In [32]:
def valid_peaks_valleys_debug(df, P, V, first):
    '''
    Remove peaks/valleys with retracement < 1/3

    Parameters
    ----------
    df: DataFrame of a security's market data
    P: peaks
    V: valleys
    first: 'P' if first peak is before first valley, else 'V'
    '''
    
    print('S: valid_peaks_valleys'.upper())
    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}\n')
    close = df.Close
    P, V = order_peaks_valleys(df, P, V, first)
    PV = sorted(P + V)
    n = len(PV)
    # not sure d = PV[3] is correct
    a, b, c, d = PV[0], PV[1], PV[2], PV[3]
    d1 = abs(close[b] - close[a])
    d2 = abs(close[c] - close[b])
    lst = P if first == 'P' else V
    pv = {'P', 'V'}
    second = list(pv - {first})[0]
    third = first if c in lst else list(pv - {first})[0]
    valid = [(a, first), (b, second), (c, third)]
    removed_vals = []
    i = 2
    
    while i < n - 2:   
        # a, b, c, d = PV[i - 2], PV[i - 1], PV[i], PV[i + 1]
        x = first if c in lst else list(pv - {first})[0]
        # x = first if i % 2 == 0 else list(pv - {first})[0]
        y = list(pv - {x})[0] 
        r = d2 / d1
        print(f'{i}. {df.index[b]} - {df.index[c]} \nc: ({x}, {close[c]:.2f})')
        print(f'd1: {close[b]:.2f} - {close[a]:.2f} = {d1:.2f}')
        print(f'd2: {close[c]:.2f} - {close[b]:.2f} = {d2:.2f}')
        print(f'{d2:.2f} / {d1:.2f} = {r:.2f}\n')

        if r < 1 / 3 * 0.95:
            removed_vals.append((c, x))
            rm, px = valid[-1]
            
            # makes first value in list lowest peak / highest valley
            reverse = True if y == 'V' else False
            val = sorted([(b, close[b]), (d, close[d])],
                         key=itemgetter(1), reverse=reverse)
            removed, kept = val[0][0], val[1][0]
            removed_vals.append((removed, y))
            
            print(f'REMOVED: \nvalid[-1]: {px} {df.index[rm].date()}, ({rm}, {close[rm]:.2f})')
            print(f'c - {x}, {df.index[c].date()} ({c}, {close[c]:.2f})' \
                  f'\nremoved - {y}, {df.index[removed].date()} ({removed}, {close[removed]:.2f})' \
                  f'\nkept - {y}, {df.index[kept].date()} ({kept}, {close[kept]:.2f})')
            # print(val)
            # valid.append((kept, y))
            if kept == d:
                try:
                    rm, px = valid.pop()
                except:
                    pass
                i = PV.index(kept)
                print(f'Skip \ni: {i} c: {kept} {df.index[kept].date()} ({y}, {close[kept]:.2f})')
                
                # l1 = [PV[i - 2], PV[i - 1], PV[i], PV[i + 1]]
                # l2 = [valid[-2][0], valid[-1][0], PV[i], PV[i + 1]]
                # print(f'valid: {l2}\norign: {l1}\nvalid {close[l2]}\norig {close[l1]}\n')
                
            elif kept == rm and px == y:
                print(f'Unchanged {(kept, y)}')
                i += 2
                print(f'Skip \ni: {i} c: {kept} {df.index[kept].date()} ({y}, {close[kept]:.2f})')
                # l1 = [PV[i - 2], PV[i - 1], PV[i], PV[i + 1]]
                # l2 = [valid[-2][0], valid[-1][0], PV[i], PV[i + 1]]
                # print(f'valid: {l2}\norign: {l1}\nvalid {close[l2]}\norig {close[l1]}\n')
                
            print()
        else:
            val, px = valid[-1]
            if val != c and px != x:
                # print(f'\ni: {i} c - {x}, {df.index[c].date()} ({c}, {close[c]:.2f})\n')
                valid.append((c, x))
            i += 1    
        
        print(f'valid: {valid[-3:]}\n')
        try:
            a, b, c, d = valid[-2][0], valid[-1][0], PV[i], PV[i + 1]
            d1 = abs(close[b] - close[a])
            d2 = abs(close[c] - close[b])
        except:
            pass
        
    P = []
    V = []

    for val, x in sorted(set(valid)):
        lst = P if x == 'P' else V
        lst.append(val)

    # P, V = order_peaks_valleys(df, P, V, first)
    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}\n')
    print('E: valid_peaks_valleys'.upper())

    return P, V

# PV = sorted(peaks + valleys)

In [33]:
period = 'W1'
days = TA_PERIODS[period]['days']
end = last_date
start = end - timedelta(days)

df = make_dataframe('a', period)[start:end]
close = df.Close
P = argrelextrema(close.to_numpy(), np.greater)[0].tolist() # peaks
V = argrelextrema(close.to_numpy(), np.less)[0].tolist() # valleys
first = 'P' if P[0] < V[0] else 'V'

valid_peaks_valleys_debug(df, P, V, first)

S: VALID_PEAKS_VALLEYS
first: P, peaks: 27, valleys: 27

2. 2021-04-30 00:00:00 - 2021-05-07 00:00:00 
c: (P, 133.90)
d1: 133.64 - 136.68 = 3.04
d2: 133.90 - 133.64 = 0.26
0.26 / 3.04 = 0.09

REMOVED: 
valid[-1]: P 2021-05-07, (7, 133.90)
c - P, 2021-05-07 (7, 133.90)
removed - V, 2021-04-30 (6, 133.64)
kept - V, 2021-05-14 (8, 131.15)
Skip 
i: 3 c: 8 2021-05-14 (V, 131.15)

valid: [(5, 'P'), (6, 'V')]

3. 2021-04-30 00:00:00 - 2021-05-14 00:00:00 
c: (V, 131.15)
d1: 133.64 - 136.68 = 3.04
d2: 131.15 - 133.64 = 2.49
2.49 / 3.04 = 0.82

valid: [(5, 'P'), (6, 'V')]

4. 2021-04-30 00:00:00 - 2021-05-28 00:00:00 
c: (P, 138.13)
d1: 133.64 - 136.68 = 3.04
d2: 138.13 - 133.64 = 4.49
4.49 / 3.04 = 1.48

valid: [(5, 'P'), (6, 'V'), (10, 'P')]

5. 2021-05-28 00:00:00 - 2021-06-04 00:00:00 
c: (V, 137.90)
d1: 138.13 - 133.64 = 4.49
d2: 137.90 - 138.13 = 0.23
0.23 / 4.49 = 0.05

REMOVED: 
valid[-1]: P 2021-05-28, (10, 138.13)
c - V, 2021-06-04 (11, 137.90)
removed - P, 2021-05-28 (10, 138.13)
kep

([5, 24, 35, 38, 41, 74, 77, 88], [6, 29, 37, 39, 65, 76, 80, 101])

In [34]:
def valid_peaks_valleys(df, P, V, first):
    '''
    Remove peaks/valleys with retracement < 1/3

    Parameters
    ----------
    df: DataFrame of a security's market data
    P: peaks
    V: valleys
    first: 'P' if first peak is before first valley, else 'V'
    '''

    print('S: valid_peaks_valleys'.upper())
    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}\n')

    close = df.Close
    P, V = order_peaks_valleys(df, P, V, first)
    PV = sorted(P + V)
    n = len(PV)
    # not sure d = PV[3] is correct
    a, b, c, d = PV[0], PV[1], PV[2], PV[3]
    d1 = abs(close[b] - close[a])
    d2 = abs(close[c] - close[b])
    lst = P if first == 'P' else V
    pv = {'P', 'V'}
    second = list(pv - {first})[0]
    third = first if c in lst else list(pv - {first})[0]
    valid = [(a, first), (b, second), (c, third)]
    removed_vals = []
    i = 2
    
    while i < n - 2:   
        x = first if c in lst else list(pv - {first})[0]
        y = list(pv - {x})[0] 
        r = d2 / d1

        if r < 1 / 3 * 0.95:
            removed_vals.append((c, x))
            rm, px = valid[-1]
            reverse = True if y == 'V' else False
            val = sorted([(b, close[b]), (d, close[d])],
                         key=itemgetter(1), reverse=reverse)
            removed, kept = val[0][0], val[1][0]
            removed_vals.append((removed, y))
            if kept == d:
                i = PV.index(kept)
                try:
                    valid.pop()
                except:
                    pass
            elif kept == rm and px == y:
                i += 2
        else:
            val, px = valid[-1]
            if val != c and px != x:
                valid.append((c, x))
            i += 1    
        try:
            a, b = valid[-2][0], valid[-1][0]
            c, d = PV[i], PV[i + 1]
            d1 = abs(close[b] - close[a])
            d2 = abs(close[c] - close[b])
        except:
            pass
    P = []
    V = []

    for val, x in sorted(set(valid)):
        lst = P if x == 'P' else V
        lst.append(val)

    return P, V

In [11]:
def trend_changepoints_debug(df):
    close = df.Close
    P = argrelextrema(close.to_numpy(), np.greater)[0].tolist() # peaks
    V = argrelextrema(close.to_numpy(), np.less)[0].tolist() # valleys
    chg = {k: {'start': [], 'end': []} for k in ['up', 'down', 'ranging']} # trend changepoints
    t = None # trend
    first = 'P' if P[0] < V[0] else 'V'
    print(f'first: {first}, peaks: {len(P)}, valleys: {len(V)}\n')
    # print('P:', P)
    # print('V:', V)
    P, V = valid_peaks_valleys(df, P, V, first)
    x, _, pv = min((P[-1], len(P), 'P'), (V[-1], len(V), 'V'), key=itemgetter(1))
    lst = P if pv == 'P' else V
    print(f'peaks: {len(P)}, valleys: {len(V)}\n')

    for p, v in zip_longest(P[2::2], V[2::2], fillvalue=x):
        ix = p if pv == 'P' else v
        i = lst.index(ix)
        g, h = i - 2, i - 1
        a, b, c = P[g], P[h], p
        d, e, f = V[g], V[h], v
        p0, p1, p2, v0, v1, v2 = close[[a,b,c,d,e,f]]
        curr_p, curr_v = [p0, p1, p2], [v0, v1, v2]
        p_data = {'Peaks': curr_p}
        v_data = {'Valleys': curr_v}
        p_dates = df.index[[a,b,c]].to_list()
        v_dates = df.index[[d,e,f]].to_list()
        p_df = pd.DataFrame(p_data, index=p_dates)
        v_df = pd.DataFrame(v_data, index=v_dates)
        pv_df = pd.concat([p_df, v_df]).sort_index().fillna('-')
        pv_df.index = [x.strftime('%d-%m-%y %H:%M') for x in pv_df.index]
        dates = sorted([df.index[p], df.index[v]])
        dates = [x.strftime("%d-%m-%y %H:%M") for x in dates]
        s = f'\n{i}. {dates} '
        s = s + t if t else s
        print(s)
        print(f'{i}. {pv} - ix: {df.index[ix].strftime("%d-%m-%y %H:%M")}')
        st_txt = '-' * 30 + '\nStart'

        if p2 > p1 > p0 and v2 > v1 > v0: # Uptrend
            print(f'{i}. Uptrend')
            if t and t != 'up':
                print(f'End {t} - {df.index[d].strftime("%d-%m-%y %H:%M")}')
                print(f'{st_txt} {df.index[d].strftime("%d-%m-%y %H:%M")}\n')
                chg[t]['end'].append(d)
                chg['up']['start'].append(d)
                pv_df['Trend'] = ['up'] * pv_df.shape[0]
                print(f'{pv_df}\n')
            t = 'up'
        elif v2 < v1 < v0 and p2 < p1 < p0: # Downtrend
            print(f'{i}. Downtrend')
            if t and t != 'down':
                print(f'End {t} - {df.index[a].strftime("%d-%m-%y %H:%M")}')
                print(f'{st_txt} {df.index[a].strftime("%d-%m-%y %H:%M")}\n')
                chg[t]['end'].append(a)
                chg['down']['start'].append(a)
                pv_df['Trend'] = ['down'] * pv_df.shape[0]
                print(f'{pv_df}\n')
            t = 'down'
        else:
            print(f'{i}. Ranging')
            if t in ('up', 'down'): # Ranging
                ix = [a, b] if t == 'up' else [e, d]
                ix = ix[0] if pv == 'P' else ix[1]
                chg[t]['end'].append(ix)
                chg['ranging']['start'].append(ix)
                pv_df['Trend'] = ['ranging'] * pv_df.shape[0]
                print(f'{pv_df}\n')
                print(f'End {t.title()} {df.index[ix].strftime("%d-%m-%y %H:%M")}\n')
            t = 'ranging'
        
    return chg, P, V

ticker = 'a'
print(ticker.upper())
period = 'D1'
days = TA_PERIODS[period]['days']
end = last_date
start = last_date - timedelta(days)
end = end + timedelta(1) if period.endswith('m') else end
df = make_dataframe(ticker, period=period)[start:end]
chg, peaks, valleys = trend_changepoints_debug(df)
print('Changepoints:')
pprint(chg)

A
first: V, peaks: 32, valleys: 33

S: VALID_PEAKS_VALLEYS
first: V, peaks: 32, valleys: 33

peaks: 20, valleys: 21


2. ['11-10-22 00:00', '13-10-22 00:00'] 
2. P - ix: 13-10-22 00:00
2. Ranging

4. ['20-10-22 00:00', '01-11-22 00:00'] ranging
4. P - ix: 01-11-22 00:00
4. Uptrend
End ranging - 11-10-22 00:00
------------------------------
Start 11-10-22 00:00

                Peaks Valleys Trend
11-10-22 00:00      -  125.64    up
13-10-22 00:00 127.90       -    up
14-10-22 00:00      -  125.70    up
18-10-22 00:00 132.30       -    up
20-10-22 00:00      -  125.94    up
01-11-22 00:00 140.89       -    up


6. ['09-11-22 00:00', '11-11-22 00:00'] up
6. P - ix: 11-11-22 00:00
6. Ranging
                Peaks Valleys    Trend
20-10-22 00:00      -  125.94  ranging
01-11-22 00:00 140.89       -  ranging
03-11-22 00:00      -  134.46  ranging
08-11-22 00:00 138.75       -  ranging
09-11-22 00:00      -  137.00  ranging
11-11-22 00:00 148.31       -  ranging

End Up 01-11-22 00:00


8. [

In [35]:
def trend_changepoints(df):
    close = df.Close
    P = argrelextrema(close.to_numpy(), np.greater)[0].tolist() # peaks
    V = argrelextrema(close.to_numpy(), np.less)[0].tolist() # valleys
    chg = {k: {'start': [], 'end': []} for k in ['up', 'down', 'ranging']} # trend changepoints
    t = None # trend
    first = 'P' if P[0] < V[0] else 'V'
    P, V = valid_peaks_valleys(df, P, V, first)
    x, _, pv = min((P[-1], len(P), 'P'), (V[-1], len(V), 'V'), key=itemgetter(1))
    lst = P if pv == 'P' else V
    
    # Identify intermediate trend changepoints comprising 5 moves,
    # i.e., 3 trending, 2 retracements 
    for p, v in zip_longest(P[2::2], V[2::2], fillvalue=x):
        ix = p if pv == 'P' else v
        i = lst.index(ix)
        g, h = i - 2, i - 1
        a, b, c = P[g], P[h], p
        d, e, f = V[g], V[h], v
        p0, p1, p2, v0, v1, v2 = close[[a,b,c,d,e,f]]
        if p2 > p1 > p0 and v2 > v1 > v0: # Uptrend
            if t and t != 'up':
                chg[t]['end'].append(d)
                chg['up']['start'].append(d)
            t = 'up'
        elif v2 < v1 < v0 and p2 < p1 < p0: # Downtrend
            if t and t != 'down':
                chg[t]['end'].append(a)
                chg['down']['start'].append(a)
            t = 'down'
        else:
            if t in ('up', 'down'): # Ranging
                ix = [a, b] if t == 'up' else [e, d]
                ix = ix[0] if pv == 'P' else ix[1]
                chg[t]['end'].append(ix)
                chg['ranging']['start'].append(ix)
            t = 'ranging'
        
    return chg, P, V

In [13]:
peaks_dates = [dt.strftime(df.iloc[i].name, '%d-%m-%y') for i in peaks]
valleys_dates = [dt.strftime(df.iloc[i].name, '%d-%m-%y') for i in valleys]
peaks_dates

['28-09-22',
 '05-10-22',
 '13-10-22',
 '18-10-22',
 '01-11-22',
 '08-11-22',
 '11-11-22',
 '18-11-22',
 '25-11-22',
 '01-12-22',
 '08-12-22',
 '13-12-22',
 '29-12-22',
 '05-01-23',
 '11-01-23',
 '23-01-23',
 '02-02-23',
 '07-02-23',
 '13-02-23',
 '03-03-23']

In [36]:
def plot_trends(ticker, period='D1', graph='line', bgcolor='slategray'):
    period_d = TA_PERIODS[period]
    days = period_d['days']
    end = last_date
    start = last_date - timedelta(days)
    end = end + timedelta(1) if period.endswith('m') else end
    df = make_dataframe(ticker, period)[start:end]
    chg, peaks, valleys = trend_changepoints(df)
    P = argrelextrema(df.Close.to_numpy(), np.greater)[0].tolist() # peaks
    V = argrelextrema(df.Close.to_numpy(), np.less)[0].tolist() # valleys
    first = 'P' if P[0] < V[0] else 'V'
    removed_PV = order_peaks_valleys_plot(df, P, V, first)

    data = []
    nrows = 1
    fig = make_subplots(rows=nrows, cols=1,
                        shared_xaxes=True, 
                        vertical_spacing=0.05,
                        subplot_titles=[''] * nrows, 
                        )

    if graph == 'candlestick':
        cs = go.Candlestick(x=df.index, 
                            open=df['Open'], 
                            high=df['High'],
                            low=df['Low'], 
                            close=df['Close'],
                            name=ticker)
        cs.increasing.fillcolor = 'green'
        cs.increasing.line.color = 'darkgreen'
        cs.decreasing.fillcolor = 'red'
        cs.decreasing.line.color = 'indianred'
        data.append(cs)
    else:   
        line = go.Scatter(x=df.index, 
                          y=df.Close, 
                          name='Close', 
                          mode='lines',
                          line_color='blue', 
                          opacity=0.5, 
                          connectgaps=True)
        data.append(line)
    
    pv_d = {'Peaks': {'color': 'orange', 'vals': P},
            'Valleys': {'color': 'red', 'vals': V}}

    # show all peaks & valleys    
    for k, v in pv_d.items():
        X = v['vals']
        pv = go.Scatter(x=df.Close[X].index,
                        y=df.Close[X],
                        name=k,
                        mode='markers',
                        marker=dict(symbol='x', color=v['color']), 
                        opacity=0.5)
        data.append(pv)
    
    # show valid peaks & valleys
    # for x in [peaks, valleys]:
    PV = sorted(peaks + valleys)
    pv = go.Scatter(x=df.Close[PV].index,
                    y=df.Close[PV],
                    name='valid_PV',
                    mode='markers',
                    marker=dict(symbol='circle-open', 
                                color='darkgreen', 
                                size=10, 
                                line_width=2.75), 
                    opacity=0.5)
    data.append(pv)

    # show removed peaks/valleys
    pv = go.Scatter(x=df.Close[removed_PV].index,
                    y=df.Close[removed_PV],
                    name='removed_PV',
                    mode='markers',
                    marker=dict(symbol='circle-x', 
                                color='red', 
                                size=10, 
                                line_width=2.75), 
                    opacity=0.5)
    data.append(pv)

    pos = [1] * len(data) # position to add rows, cols in subplot 
    fig.add_traces(data=data, rows=pos, cols=pos) 
    
    chg_start = []
    chg_end = []

    for k, v in chg.items():
        for k1, v1 in v.items():
            if k1 == 'start':
                chg_start.extend(v1)
                line_dash = 'dot'
                lw = 1
            else:
                chg_end.extend(v1)
                line_dash = 'solid'
                lw = 0.75
            for x in v1:
                if k != 'ranging':
                    color = 'gold' if k == 'up' else 'darkred'
                    # s = {'up', 'down'}
                    # t = {'start', 'end'}
                    # a = str(list(s - {k})[0])
                    # b = str(list(t - {k1})[0])
                    # if x in chg[a][b]:
                    #     color = 'green'
                    #     line_dash = 'dashdot'
                    fig.add_vline(x=df.index[x], line_color=color, line_width=lw, line_dash=line_dash)
                elif k1 != 'end':
                    color = 'violet'
                    line_dash = 'solid'
                    fig.add_vline(x=df.index[x], line_color=color, line_width=lw, line_dash=line_dash)

    us_holidays = pd.to_datetime(list(holidays.US(range(start.year, end.year + 1)).keys()))
    rangebreaks = []
    rangeselector = []

    if period == 'M1':
        rangeselector = dict(buttons=[
                                dict(count=1, label="YTD", step="year", stepmode="todate"),
                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                dict(count=2, label="2y", step="year", stepmode="backward"),
                                dict(count=3, label="3y", step="year", stepmode="backward"),
                                dict(step="all")
                                ])
    elif period == 'W1':
        rangeselector = dict(buttons=[
                                dict(count=1, label="YTD", step="year", stepmode="todate"),
                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                dict(step="all")
                                ])  
    elif period == 'D1':
        rangebreaks = [dict(bounds=["sat", "mon"])]
    else:
        us_holidays += pd.offsets.Hour(9) + pd.offsets.Minute(30)
        rangebreaks = [dict(bounds=[16, 9.5], pattern="hour"), 
                       dict(bounds=["sat", "mon"])]

    if rangebreaks:
        us_holidays = pd.to_datetime(sorted(list(set(us_holidays) - set(df.index))))
        rangebreaks.append(dict(values=us_holidays))
        fig.update_xaxes(rangebreaks=rangebreaks)

    if rangeselector:
        fig.update_layout(xaxis1=dict(rangeselector=rangeselector))

    # pprint(fig.layout)

    cname = SPY_info_df.loc[ticker.upper(), 'Security']
    title = f'{cname} ({ticker.upper()}) - {period}'

    fig.update_layout(title=dict(text=title, xanchor='left'), paper_bgcolor=bgcolor, plot_bgcolor=bgcolor)
    fig.layout.xaxis.rangeslider.visible = False
    
    return fig

In [39]:
plot_trends('a', period='W1', bgcolor='slategray')

S: VALID_PEAKS_VALLEYS
first: P, peaks: 27, valleys: 27

	S: ORDER_PEAKS_VALLEYS

	first: P, peaks: 27, valleys: 27
	E: ORDER_PEAKS_VALLEYS


In [ ]:
    # for x, y in trendlines:
    #     fig.add_scatter(x=x,
    #                     y=y,
    #                     name='Trend',
    #                     mode='lines', 
    #                     opacity=0.5,
    #                     showlegend=False)
    # fig.add_scatter(x=close[peak_valleys].index,
    #                 y=close[peak_valleys],
    #                 name='peak_valleys',
    #                 mode='markers',
    #                 marker=dict(symbol='circle-open', color='green', size=8))

### S/R Levels

In [3]:
from sklearn import preprocessing
import time
# pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

In [4]:
end = last_date
years = 1
days = 180
start = end - timedelta(days * years)
ticker = 'aapl'
df = get_ticker_data(ticker)[start:end]

In [5]:
def fibonacci_retracement_levels(df):
    highest_swing = -1
    lowest_swing = -1
    high = df['High']
    low = df['Low']
    nr = df.shape[0]

    for i in range(1, nr - 1):
        if high[i] > high[i - 1] and high[i] > high[i + 1] \
            and (highest_swing == -1 or high[i] > high[highest_swing]):
            highest_swing = i
        if low[i] < low[i - 1] and low[i] < low[i + 1] \
            and (lowest_swing == -1 or low[i] < low[lowest_swing]):
            lowest_swing = i

    ratios = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
    levels = []
    max_level = high[highest_swing]
    min_level = low[lowest_swing]

    for ratio in ratios:
        # Uptrend
        if highest_swing > lowest_swing:
            level = max_level - (max_level - min_level) * ratio
        # Downtrend
        else:
            level = min_level + (max_level - min_level) * ratio
        levels.append(level)

    frl_data = {}
    unit = 'minutes' if df.iloc[0].name.minute != 0 else 'days'  
    prev_date = df.iloc[0].name

    for i in range(nr):
        date = df.iloc[i].name
        s_date = date.strftime('%d-%m-%y')
        l = bisect.bisect_left(levels, low[i])
        h = bisect.bisect_left(levels, high[i])
        h = h - 1 if h > 0 else h
        frls = [levels[h], levels[l]] if h != l else [levels[h]]
        # Deals with multiple levels being tested by one bar
        for frl in frls:
            if high[i] > frl and low[i] < frl:
                cum_vol = df.loc[prev_date:date, 'Volume'].sum()
                delta = date - prev_date # time it takes level to form
                delta = delta.days if unit == 'days' else delta.total_seconds() / 60
                prev_date = date
                d = {'Date': [], 'Timedelta': [], 'Volume': [], 'Tested': 0}
                frl_data.setdefault(frl, d)
                frl_data[frl]['Date'].append(s_date)
                frl_data[frl]['Timedelta'].append(delta)
                frl_data[frl]['Volume'].append(cum_vol)
                frl_data[frl]['Tested'] += 1             

    # Calculate significance of levels       
    d = {'FRL': [], 'Date': [], 'Timedelta': [], 'Volume': [], 'Tested': []}

    for k, v in frl_data.items():
        d['FRL'].append(k)
        d['Volume'].append(sum(v['Volume']))
        d['Timedelta'].append(sum(v['Timedelta']))
        d['Tested'].append(v['Tested'])
        d['Date'].append(v['Date'][-1])
        # print(v['Date'][-1])

    # NOT YET DECIDED HOW TO CALCULATE FRL SIGNAL

    return ratios, levels, frl_data


In [6]:
r, l, d = fibonacci_retracement_levels(df)
d

{155.68073529052734: {'Date': ['02-09-22',
   '06-09-22',
   '07-09-22',
   '08-09-22',
   '09-09-22',
   '13-09-22',
   '14-09-22',
   '20-09-22',
   '21-09-22',
   '28-10-22',
   '03-02-23',
   '16-02-23'],
  'Timedelta': [0, 4, 1, 1, 1, 4, 1, 4, 1, 1, 1, 13],
  'Volume': [76957800,
   150672600,
   161164400,
   172373400,
   152952600,
   295641400,
   210622000,
   351442800,
   209386600,
   273942600,
   272618900,
   742740900],
  'Tested': 12},
 148.9456159057617: {'Date': ['16-09-22',
   '23-09-22',
   '28-09-22',
   '24-10-22',
   '26-10-22',
   '27-10-22',
   '28-10-22',
   '02-11-22',
   '11-11-22',
   '14-11-22',
   '15-11-22',
   '16-11-22',
   '17-11-22',
   '21-11-22',
   '22-11-22',
   '01-12-22',
   '05-12-22',
   '13-12-22',
   '02-02-23',
   '03-02-23',
   '21-02-23',
   '22-02-23',
   '23-02-23',
   '27-02-23',
   '28-02-23'],
  'Timedelta': [2,
   2,
   5,
   3,
   2,
   1,
   0,
   5,
   1,
   3,
   1,
   1,
   1,
   4,
   1,
   1,
   4,
   1,
   1,
   0,
   5,


In [7]:
def isSupport(df, i):
    '''Returns True if value is a price support level'''

    X = df['Low']
    support = X[i] < X[i - 1] \
                and X[i] < X[i + 1] \
                and X[i + 1] < X[i + 2] \
                and X[i - 1] < X[i - 2]

    return support


def isResistance(df, i):
    '''Returns True if value is a price resistance level'''

    X = df['High']
    resistance = X[i] > X[i - 1] \
                    and X[i] > X[i + 1] \
                    and X[i + 1] > X[i + 2] \
                    and X[i - 1] > X[i - 2] 

    return resistance
    

In [8]:
def convert_to_timestamp(x):
    """Convert date objects to integers"""
    
    return time.mktime(x.timetuple())

In [14]:
def sr_levels(df):
    '''Returns key support/resistance levels for a security'''

    df = df.copy()
    df['SR Signal'] = 0
    prev_date = df.iloc[0].name
    unit = 'minutes' if prev_date.minute != 0 else 'days'  
    spt, rst = (0, 0)
    levels = []
    s_levels = []
    sr_data = {}
    many_tests = {} # dict of bars that test more than 1 level
    s = (df['High'] - df['Low']).mean()
    nr, nc = df.shape

    def isFarFromLevel(l):
        '''Returns True if price is not near a previously discovered support or resistance'''
        
        return np.sum([abs(l - x[1]) < s for x in levels]) == 0

    for i in range(2, nr):
        date = df.iloc[i].name
        s_date = date.strftime('%d-%m-%y')
        high = df['High'][i]
        low = df['Low'][i]
        close = df['Close'][i]
        new_spt = False
        new_rst = False
        sr_switch = False

        if i < nr - 2:
            if isSupport(df, i):
                if isFarFromLevel(low):
                    new_spt = True
                    spt = low
                    df.loc[date, 'Support'] = spt
                    levels.append((i, spt))
                    s_levels = sorted([x[1] for x in levels])
                    print('NS'.ljust(5), f'- {date.date()} - S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
            
            if isResistance(df, i):
                if isFarFromLevel(high):
                    new_rst = True
                    rst = high
                    df.loc[date, 'Resistance'] = rst
                    levels.append((i, rst))
                    s_levels = sorted([x[1] for x in levels])
                    print('NR'.ljust(5), f'- {date.date()} - R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f},')    

        # Switch support to resistance & vice versa
        if len(levels) > 1:
            if new_spt:
                ix = bisect.bisect(s_levels, spt)
                rst = s_levels[ix] if ix < len(s_levels) else s_levels[ix - 1]
            if new_rst:
                ix = bisect.bisect_left(s_levels, rst)
                spt = s_levels[ix - 1] if ix > 0 else s_levels[ix]
            if low > rst: # When resistance broken 
                sr_switch = True
                spt = rst
                ix = bisect.bisect(s_levels, low)
                rst = s_levels[ix] if ix < len(s_levels) else s_levels[ix - 1]
                print('R-S'.ljust(5), f'- {date.date()} - S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
            if high < spt: # When support broken 
                sr_switch = True
                rst = spt
                ix = bisect.bisect_left(s_levels, high)
                spt = s_levels[ix - 1] if ix > 0 else s_levels[ix]
                print('S-R'.ljust(5), f'- {date.date()} - R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f}')
        
        if new_rst or new_spt or sr_switch:
            cum_vol = df.loc[prev_date:date, 'Volume'].sum()
            delta = date - prev_date # time it takes level to form
            delta = delta.days if unit == 'days' else delta.total_seconds() / 60
            prev_date = date
            d = {'Date': [], 'Timedelta': [], 'Volume': [], 'SR': [], 'Tested': 0, 'Tested Date': []}
            sr_data.setdefault(spt, d)
            sr_data[spt]['Date'].append(s_date)
            sr_data[spt]['Timedelta'].append(delta)
            sr_data[spt]['Volume'].append(cum_vol)
            sr_data[spt]['SR'].append('S')
            # Prevents double-counting when support == resistance
            if spt != rst:
                sr_data.setdefault(rst, d)
                sr_data[rst]['Date'].append(s_date)
                sr_data[rst]['Timedelta'].append(delta)
                sr_data[rst]['Volume'].append(cum_vol)
                sr_data[rst]['SR'].append('R')
                                  
        if spt:
            if close < spt:
                df.loc[date, 'SR Signal'] = 1 # Generate signal
            # Check if S/R levels are tested       
            if high > spt and low < spt:
                sr_data[spt]['Tested'] += 1
                sr_data[spt]['Tested Date'].append(s_date)
                print('ST'.ljust(5), f'- {date.date()} - S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                ix = bisect.bisect_left(s_levels, spt)
                n_spt = s_levels[ix - 1] if ix > 0 else s_levels[ix]    
                while low < n_spt and spt != rst and spt != n_spt:
                    print(f'SH-SL - {date.date()} - NS: {n_spt:.2f}, S: {spt:.2f}, R: {rst:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                    rst = spt
                    spt = n_spt
                    many_tests.setdefault(i, set()).union([spt, rst])
                    if ix > 0:
                        ix -= 1
                        n_spt = s_levels[ix]
                        sr_data[n_spt]['Date'].append(s_date)
                        sr_data[n_spt]['Timedelta'].append(delta)
                        sr_data[n_spt]['Volume'].append(cum_vol)
                        sr_data[n_spt]['SR'].append('S')            
   
        if rst:
            if close > rst:
                df.loc[date, 'SR Signal'] = 1 # Generate signal
            # Check if S/R levels are tested       
            if high > rst and low < rst:
                if spt != rst: # Prevents double-counting
                    sr_data[rst]['Tested'] += 1
                    sr_data[rst]['Tested Date'].append(s_date)
                    print('RT'.ljust(5), f'- {date.date()} - R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                    ix = bisect.bisect(s_levels, rst)
                    n_rst = s_levels[ix] if ix < len(s_levels) else s_levels[ix - 1]
                    while high > n_rst and spt != rst and rst != n_rst:
                        print(f'RL-RH - {date.date()} - NR: {n_rst:.2f}, R: {rst:.2f}, S: {spt:.2f}, hi: {high:.2f}, lo: {low:.2f}')
                        spt = rst
                        rst = n_rst
                        many_tests.setdefault(i, set()).union([spt, rst])
                        if ix < len(s_levels) - 1:
                            ix += 1
                            # print(f'ix: {ix}, {s_levels}')
                            n_rst = s_levels[ix]
                            sr_data[n_rst]['Date'].append(s_date)
                            sr_data[n_rst]['Timedelta'].append(delta)
                            sr_data[n_rst]['Volume'].append(cum_vol)
                            sr_data[n_rst]['SR'].append('R')        

        if spt and rst: 
            df.loc[date:, 'Support'] = spt
            df.loc[date:, 'Resistance'] = rst

    del sr_data[0]     
    d = {'SR Level': [], 'Volume': [], 'Timedelta': [], 'Tested': [], 'Date': []}

    for k, v in sr_data.items():
        d['SR Level'].append(k)
        d['Volume'].append(sum(v['Volume']))
        d['Timedelta'].append(sum(v['Timedelta']))
        d['Tested'].append(v['Tested'])
        d['Date'].append(v['Date'][-1])

    ix = 'SR Level'
    sr_df = pd.DataFrame(d, index=d[ix]).drop(columns=ix)
    sr_df['Date'] = pd.to_datetime(sr_df['Date'])
    sr_df['Date'] = sr_df['Date'].apply(convert_to_timestamp)
    scaler = preprocessing.MinMaxScaler(feature_range=(1, 5))
    sd = scaler.fit_transform(sr_df)
    scaled_df = pd.DataFrame(sd, columns=sr_df.columns, index=d[ix])
    scaled_df['Signal'] = scaled_df.mean(axis=1)
    # print(sr_df)
    # print(scaled_df)
    
    # Make 'SR Signal' last column
    cols = list(df.columns)
    cols.append(cols.pop(cols.index('SR Signal')))
    df = df[cols]
    nr, nc = df.shape
    j = nc - 1 # 'SR Signal' column num

    # Assign significance to signals by level
    for i in range(nr):
        if df['SR Signal'][i]:
            if i in many_tests:
                signal = 0
                for l in many_tests[i]:
                    signal += scaled_df.loc[l, 'Signal']
            else:
                if df['Close'][i] > df['Resistance'][i]:
                    l = df['Resistance'][i]
                if df['Close'][i] < df['Support'][i]:
                    l = df['Support'][i]
                signal = scaled_df.loc[l, 'Signal']
 
            df.iloc[i, j] = signal        
    
    return levels, df, sr_data

In [15]:
# sr_data.clear()
levels, ta_df, sr_data = sr_levels(df)
# levels

NS    - 2022-09-08 - S: 152.68, R: 0.00, hi: 156.36, lo: 152.68
NR    - 2022-09-12 - R: 164.26, S: 152.68, hi: 164.26, lo: 159.30,
ST    - 2022-09-15 - S: 152.68, R: 164.26, hi: 155.24, lo: 151.38
NS    - 2022-09-16 - S: 148.37, R: 164.26, hi: 151.35, lo: 148.37
RT    - 2022-09-19 - R: 152.68, S: 148.37, hi: 154.56, lo: 149.10
R-S   - 2022-09-20 - S: 152.68, R: 164.26, hi: 158.08, lo: 153.08
NR    - 2022-09-21 - R: 158.74, S: 152.68, hi: 158.74, lo: 153.60,
ST    - 2022-09-22 - S: 152.68, R: 158.74, hi: 154.47, lo: 150.91
S-R   - 2022-09-23 - R: 152.68, S: 148.37, hi: 151.47, lo: 148.56
RT    - 2022-09-26 - R: 152.68, S: 148.37, hi: 153.77, lo: 149.64
RT    - 2022-09-27 - R: 152.68, S: 148.37, hi: 154.72, lo: 149.95
ST    - 2022-09-28 - S: 148.37, R: 152.68, hi: 150.64, lo: 144.84
S-R   - 2022-09-29 - R: 148.37, S: 148.37, hi: 146.72, lo: 140.68
S-R   - 2022-09-30 - R: 148.37, S: 148.37, hi: 143.10, lo: 138.00
S-R   - 2022-10-03 - R: 148.37, S: 148.37, hi: 143.07, lo: 137.69
S-R   - 20

In [16]:
pprint(sr_data)

{134.3699951171875: {'Date': ['13-10-22',
                              '03-11-22',
                              '07-12-22',
                              '15-12-22',
                              '20-12-22',
                              '23-12-22',
                              '27-12-22',
                              '28-12-22',
                              '29-12-22',
                              '30-12-22',
                              '03-01-23',
                              '04-01-23',
                              '05-01-23',
                              '06-01-23',
                              '09-01-23',
                              '10-01-23',
                              '11-01-23',
                              '12-01-23',
                              '18-01-23'],
                     'SR': ['S',
                            'S',
                            'S',
                            'S',
                            'S',
                            'S',
   

In [26]:
d = {'SR Level': [], 'Volume': [], 'Timedelta': [], 'Tested': []}

for k, v in sr_data.items():
    d['SR Level'].append(k)
    d['Volume'].append(sum(v['Volume']))
    d['Timedelta'].append(sum(v['Timedelta']))
    d['Tested'].append(v['Tested'])

sr_df = pd.DataFrame(d, index=d['SR Level']).drop(columns='SR Level')
scaler = preprocessing.MinMaxScaler(feature_range=(1, 5))
sd = scaler.fit_transform(sr_df)
cols = ['Volume', 'Timedelta', 'Tested']
scaled_df = pd.DataFrame(sd, columns=sr_df.columns, index=d['SR Level'])
scaled_df['Signal'] = scaled_df.mean(axis=1)
print(sr_df.head(), '\n')
print(scaled_df)

            Volume  Timedelta  Tested
152.68  4975022300         70      16
164.26   609351400          8       0
148.37  8030617400        102      22
158.74  2146815800         31       0
134.37  4587985500         53       8 

        Volume  Timedelta  Tested  Signal
152.68    3.35       3.64    3.91    3.63
164.26    1.00       1.00    1.00    1.00
148.37    5.00       5.00    5.00    5.00
158.74    1.83       1.98    1.00    1.60
134.37    3.14       2.91    2.45    2.84
144.13    2.53       3.04    3.18    2.92


## MA Signals

- MA Crossovers, i.e. MA5/MA25, etc...
- Identify when moving averages converge

In [34]:
def calc_slope(x):
    slope = np.polyfit(range(len(x)), x, 1)[0] 
    
    return slope

In [98]:
def calculate_ma_signals(df, MAs):
    for i, ma in enumerate(MAs):
        ix = df.iloc[ma].name
        df[f'MA{ma}'] = df['Close'].rolling(ma).mean()
        df[f'MA{ma} Slope'] = df[f'Close'].rolling(ma).apply(calc_slope)
        # df[f'MA{ma} Slope Diff'] = df[f'MA{ma} Slope'].diff()
        df.loc[ix:, f'MA{ma} Signal'] = np.where(df['Close'][ma:] > df[f'MA{ma}'][ma:], 1, 0)
        df[f'MA{ma} Crossover'] = df[f'MA{ma} Signal'].diff()
        df.drop(columns=f'MA{ma} Signal', inplace=True)

    df.loc[:, 'MA Std. Dev'] = df.loc[:, [f'MA{ma}' for ma in MAs]].std(axis=1)
    
    return df

In [100]:
MAs = [10, 20, 50]
ma_df = calculate_ma_signals(ta_df, MAs)
# ma_df.loc['2022-09-12':'2022-11-12', ['MA10 Slope',	'MA10 Slope Diff', 'MA20 Slope', 'MA50 Slope']]
ma_df.loc['2022-09-12':'2022-11-12', ['MA10', 'MA20', 'MA50', 'MA Std. Dev']]

,MA10,MA20,MA50,MA Std. Dev
2022-09-12,157.70,NaN,NaN,NaN
2022-09-13,156.95,NaN,NaN,NaN
2022-09-14,156.59,NaN,NaN,NaN
2022-09-15,156.10,NaN,NaN,NaN
2022-09-16,155.38,NaN,NaN,NaN
2022-09-19,155.24,NaN,NaN,NaN
2022-09-20,155.48,NaN,NaN,NaN
2022-09-21,155.26,NaN,NaN,NaN
2022-09-22,155.09,NaN,NaN,NaN
2022-09-23,154.39,NaN,NaN,NaN


In [58]:
math.atan2(ma_df['MA10'].values, ma_df.index.values)

TypeError: only size-1 arrays can be converted to Python scalars

In [74]:
ma_df.loc['2022-10-03':'2022-10-04', ['MA10', 'MA10 Slope', 'MA20', 'MA20 Slope']]

,MA10,MA10 Slope,MA20,MA20 Slope
2022-10-03,148.93,-0.73,152.09,NaN
2022-10-04,147.85,-0.83,151.67,NaN


In [75]:
x = ma_df.loc['2022-10-03':'2022-10-04', 'MA10']
slope = np.polyfit(range(len(x)), x, 1)[0] 
slope


-1.0799987792968864

### Bollinger Bands

In [17]:
from talib import BBANDS, MACD, RSI

In [18]:
up, mid, down = BBANDS(df['Close'], timeperiod=21, nbdevup=2, nbdevdn=2, matype=0)
bands = {'BB Up': up, 'BB Mid': mid, 'BB Down': down}
bb_df = pd.DataFrame(bands)
bb_df

,BB Up,BB Mid,BB Down
2022-08-29,NaN,NaN,NaN
2022-08-30,NaN,NaN,NaN
2022-08-31,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN
2022-09-02,NaN,NaN,NaN
...,...,...,...
2023-02-17,159.14,148.58,138.01
2023-02-21,158.50,149.08,139.66
2023-02-22,158.18,149.45,140.73
2023-02-23,157.94,149.78,141.62


In [131]:
open.rolling(5, center=True).mean()

Datetime
2023-02-13 09:30:00-05:00      NaN
2023-02-13 09:35:00-05:00      NaN
2023-02-13 09:40:00-05:00   158.76
2023-02-13 09:45:00-05:00   158.66
2023-02-13 09:50:00-05:00   158.61
                             ...  
2023-02-16 15:40:00-05:00   160.52
2023-02-16 15:45:00-05:00   160.51
2023-02-16 15:50:00-05:00   160.51
2023-02-16 15:55:00-05:00      NaN
2023-02-16 16:00:00-05:00      NaN
Name: Open, Length: 313, dtype: float64

In [16]:
from talib import BBANDS, MACD, RSI

In [17]:
up, mid, low = BBANDS(df.Close, timeperiod=21, nbdevup=2, nbdevdn=2, matype=0)

In [21]:
# df[['BB Up', 'BB Mid', 'BB Down']] = up, mid, low
df['BB Up'] = up
df['BB Mid'] = mid
df['BB Down'] = low
df

,Open,High,Low,Close,Adj Close,Volume,BB Up,BB Mid,BB Down
Datetime,,,,,,,,,
2023-02-13 09:30:00-05:00,158.89,159.91,158.64,158.72,158.72,19034,NaN,NaN,NaN
2023-02-13 09:35:00-05:00,158.80,159.34,158.80,159.11,159.11,5547,NaN,NaN,NaN
2023-02-13 09:40:00-05:00,158.95,159.05,158.58,158.78,158.78,9484,NaN,NaN,NaN
2023-02-13 09:45:00-05:00,158.70,158.78,158.23,158.43,158.43,51620,NaN,NaN,NaN
2023-02-13 09:50:00-05:00,158.44,158.70,158.37,158.51,158.51,11596,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2023-02-16 15:40:00-05:00,160.51,160.70,160.24,160.67,160.67,32917,160.70,160.33,159.96
2023-02-16 15:45:00-05:00,160.66,160.69,160.16,160.31,160.31,52181,160.67,160.35,160.02
2023-02-16 15:50:00-05:00,160.35,160.85,160.34,160.59,160.59,79573,160.68,160.37,160.06


In [23]:
rsi = RSI(df.Close, timeperiod=14)
rsi.index

DatetimeIndex(['2023-02-13 09:30:00-05:00', '2023-02-13 09:35:00-05:00',
               '2023-02-13 09:40:00-05:00', '2023-02-13 09:45:00-05:00',
               '2023-02-13 09:50:00-05:00', '2023-02-13 09:55:00-05:00',
               '2023-02-13 10:00:00-05:00', '2023-02-13 10:05:00-05:00',
               '2023-02-13 10:10:00-05:00', '2023-02-13 10:15:00-05:00',
               ...
               '2023-02-16 15:15:00-05:00', '2023-02-16 15:20:00-05:00',
               '2023-02-16 15:25:00-05:00', '2023-02-16 15:30:00-05:00',
               '2023-02-16 15:35:00-05:00', '2023-02-16 15:40:00-05:00',
               '2023-02-16 15:45:00-05:00', '2023-02-16 15:50:00-05:00',
               '2023-02-16 15:55:00-05:00', '2023-02-16 16:00:00-05:00'],
              dtype='datetime64[ns, pytz.FixedOffset(-300)]', name='Datetime', length=313, freq=None)

In [123]:
macd, macdsignal, macdhist = MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)